# PitViper Notebook Report

Ce notebook a été généré automatiquement par l'outil PitViper.

Il peut être utilisé de deux manières :

1. En utilisant les fonctions déjà crées et présentes dans les cellules suivantes.

2. En créant de nouvelles cellules et en y écrivant du code python3.

Les graphiques sont générés à l'aide de la bibliothèque python [Altair](https://altair-viz.github.io/index.html). Il est possible de télécharger chaque graphique au format SVG depuis le menu déroulant présent en haut à droite de chaque graphique.

La cellule suivante permet d'appeler les fonctions déjà crées pour la visualisation des résultats.

In [1]:
import sys
import os


modules_path = ['workflow/notebooks/', "../../../workflow/notebooks/"]
for module in modules_path:
    module_path = os.path.abspath(os.path.join(module))
    if module_path not in sys.path:
        sys.path.append(module_path)


from functions_pitviper_nb import * 

import time

alt.renderers.enable('html')

# For Snakemake
switch = True
while os.path.basename(os.getcwd()) != "PitViper":
    if switch:
        switch = False
        %cd ../../
    else:
        %cd ../

print('Working directory: ', os.getcwd())

/media/paularthur/data/Projets/Screens/PitViper/PitViper
Working directory:  /media/paularthur/data/Projets/Screens/PitViper/PitViper


Lors de la création automatique du rapport, les deux cellule suivante sont utilisées pour définir les dossiers contenant les résultats des différents outils utilisés.

Il est recommandé de ne pas les modifier.

In [2]:
# Parameters
mageck_mle_outputs = "results/development/MAGeCK_MLE/"
mageck_rra_outputs = "results/development/MAGeCK_RRA/"
bagel_outputs = "results/development/BAGEL/"
crisphiermix_outputs = "results/development/CRISPhieRmix/"

In [3]:
results_paths = [mageck_mle_outputs, mageck_rra_outputs, bagel_outputs, crisphiermix_outputs]

def configuration(results_paths):
    
    results_path_to_use = []
    for path in results_paths:
        if os.path.exists(path):
            results_path_to_use.append(path)
    return results_path_to_use

results = create_results_pitviper(configuration(results_paths))

## Quality Control

In [4]:
show_mapping_qc('CRISPRi_countFrom_bam')

,Label,Reads,Mapped,Percentage,Zerocounts,GiniIndex
0,J4_rep1,9419086,9127498,0.969000,147,0.195100
4,J4_rep2,10210515,9892397,0.968800,96,0.177700
8,J4_rep3,10338534,10012818,0.968500,93,0.175500
1,J11_rep1,7844354,7602610,0.969200,156,0.194100
5,J11_rep2,9625449,9327159,0.969000,163,0.184300
9,J11_rep3,11782106,11423343,0.969600,308,0.197400
2,J18_rep1,9483453,9194535,0.969500,228,0.200900
6,J18_rep2,10793945,10461133,0.969200,256,0.194200
10,J18_rep3,11493016,11135161,0.968900,225,0.183500
3,J25_rep1,14000927,13561366,0.968600,270,0.200100


In [5]:
show_read_count_distribution('CRISPRi_countFrom_bam')

alt.Chart(...)

In [6]:
# gene_selection_table(results)

In [7]:
test_plotting(results['MAGeCK_MLE'])

interactive(children=(Dropdown(description='baseline', options=('M07e-J11', 'M07e-J18', 'M07e-J4'), value='M07…

La fonction `sgRNA_accross_conditions` permet de visualiser l'évolution du score des sgRNAs prédit par l'outil MAGeCK d'une *feature* d'intérêt dans une condition donnée.

In [8]:
sgRNA_accross_conditions(results['MAGeCK_RRA'])

interactive(children=(Text(value='MYC', description='Feature:', placeholder='Feature to show...'), Dropdown(de…

## Gene essentiality analysis

Les fonctions suivantes permettent de visualiser l'évolution du score des `features` pour chaque outil.

### MAGeCK MLE - Feature accross conditions

In [9]:
feature_accros_conditions(results['MAGeCK_MLE'])

interactive(children=(Dropdown(description='baseline', options=('M07e-J11', 'M07e-J18', 'M07e-J4'), value='M07…

### MAGeCK RRA - Feature accross conditions

In [10]:
feature_accros_conditions(results['MAGeCK_RRA'])

interactive(children=(Dropdown(description='baseline', options=('M07e-J11', 'M07e-J18', 'M07e-J4'), value='M07…

### CRISPhieRmix - Feature accross conditions

In [11]:
feature_accros_conditions(results['CRISPhieRmix'])

interactive(children=(Dropdown(description='baseline', options=('M07e-J11', 'M07e-J18', 'M07e-J4'), value='M07…

### BAGEL - Feature accross conditions

In [12]:
feature_accros_conditions(results['BAGEL'])

interactive(children=(Dropdown(description='baseline', options=('M07e-J11', 'M07e-J18', 'M07e-J4'), value='M07…

## Annotations

### EnrichR

In [13]:
enrichr_plots(results)

interactive(children=(Dropdown(description='tool', options=('MAGeCK_MLE', 'MAGeCK_RRA', 'BAGEL', 'CRISPhieRmix…

In [14]:
from sklearn.preprocessing import StandardScaler

from sklearn.decomposition import PCA

import matplotlib as figure

def pca(token):
    path_qc = "./data/{token}/screen.count.txt".format(token=token)
    table = pd.read_csv(path_qc, sep='\t')
        
    table.iloc[:, 2:] = table.iloc[:, 2:] +1 
    table.iloc[:, 2:] = table.iloc[:, 2:].apply(np.log2)

    features = list(table.columns[2:])
    
    x = table.loc[:, features].values
    
    x = StandardScaler().fit_transform(x)
    
    pca = PCA(n_components=2)

    principalComponents = pca.fit_transform(x)

    principalDf = pd.DataFrame(data = principalComponents
             , columns = ['principal component 1', 'principal component 2'])
    
#     fig = plt.figure.Figure(figsize = (8,8))
#     ax = fig.add_subplot(1,1,1)
#     fig = ax.scatter(principalDf)

    return principalDf

    
pca('CRISPRi_countFrom_bam')

,principal component 1,principal component 2
0,-1.726843,-0.291542
1,-6.779774,-0.201359
2,2.710999,-0.060509
3,0.397749,0.282131
4,0.492596,-0.043351
...,...,...
7451,-9.847964,0.085482
7452,-9.847964,0.085482
7453,-9.847964,0.085482
7454,-9.847964,0.085482
